Task 3<br>
<ol>
<li>Create custom dataloader
<li>Custom CNN
<li>Custom training and evaluation function
</ol>

Data can be found at 
<a href="https://drive.google.com/file/d/1c9R_WO0ttNFSYXLG7Zap0ToBASLLq6Rb/view?usp=sharing">here</a>

As a label to our data, we ought to use action field<br>

First import the python tools

In [1]:
import matplotlib.pyplot as plt
import torch 

import numpy as np

import torchvision
import torch.nn as nn
import torch.nn.functional as f

from os import *
import random
from torch.utils.data import Dataset

Here I define all labels for fields in dataset
Code below was provided by my college 

In [2]:
classes = [
        'frame_num', 
        'action', 
        'team_controlling_ball', 
        'ball_x', 
        'ball_y', 
        'num_players_team1', 
        'num_players_team2', 
        'scene', 
        'no_team1_players',
    ]


for i in range(11):
    classes.append('team1_player' + str(i) + 'x')
    classes.append('team1_player' + str(i) + 'y')

classes.append('no_team1_players')

for i in range(11):
    classes.append('team2_player' + str(i) + 'x')
    classes.append('team2_player' + str(i) + 'y')

Define custom dataloader 

In [3]:
class my_dataloader(Dataset):
        def __init__(self, input_file):
            
            dataset = np.load(input_file)
            self.length = len(dataset) - 1
            
            try:
                mkdir("data")
            except:
                print("Out folder arledy exists skiping")
                return
            
            i = 0
            for data in dataset:
                file = "./data/"+ str(i) + ".npy"
                np.save(file, data)
                i+=1
        
        def __len__(self):
            return self.length
        
        def __getitem__(self, i):
            if i == self.length:
                return StopIteration
            train = np.load("./data/"+str(random.randint(0, self.length))+".npy")
            train_data = [(float(train[3])),(float(train[4]))]
            train_data = np.array([train_data],dtype=np.float32)
            train_data = torch.from_numpy(train_data)
            train_labels = torch.from_numpy(np.array([int(train[1])]))
            return train_data, train_labels

load data from file

In [4]:
data_loader = my_dataloader("./data.npy")

FileNotFoundError: ignored

My custom neural net class, 
copied from assigment 2 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.lay1 = nn.Linear(2,20)
        self.lay2 = nn.Linear(20,20)
        self.lay3 = nn.Linear(20,41)
        
    def forward(self,x):
        
        x = f.relu(self.lay1(x))
        x = f.relu(self.lay2(x))
        
        return self.lay3(x)

net = Net()

define optimizer and loss function

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

main loop, here I feed data to my neural net,<br>
and calculate the resolut  



In [ ]:
losses = []
i = 1
current_loss = 0
for chunk in data_loader:
    train_data, train_label = chunk
    optimizer.zero_grad()
    guess = net(train_data)
    loss = criterion(guess.view(1,41), train_label)
    loss.backward()
    optimizer.step()
    current_loss += loss.item()
    i += 1
    
    if i == 2000:
        print("Current loss: " + str(current_loss / 2000))
        losses.append(current_loss/2000)
        current_loss = 0
        i = 0

plot loss funtion

In [ ]:
plt.plot(losses)
plt.show()

calculate final error

In [ ]:
correct = 0  
total = 0
with torch.no_grad():
    for i, data in enumerate(data_loader): 
        images, labels = data
        images = images
        labels = labels
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size()[0]
        correct += (predicted == labels).sum().item()
    
print("End accuracy: ", correct/total)